#  🏦 Openly Catastrophe Impact Simulator

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

DATA_DIR = '../data/'
OUTPUT_DIR = '../data/'

print("Libraries and paths set.")

## 2. Data Loading & Validation

In [ ]:
def load_and_validate(file, required_cols=None):
    """Load CSV and validate required columns, report missing, handle errors."""
    path = os.path.join(DATA_DIR, file)
    try:
        df = pd.read_csv(path)
        if required_cols:
            missing = [col for col in required_cols if col not in df.columns]
            if missing:
                raise ValueError(f"Missing columns in {file}: {missing}")
        return df
    except FileNotFoundError as e:
        raise FileNotFoundError(f"File not found at {path}") from e
    except Exception as e:
        raise ValueError(f"Error loading {file}: {str(e)}") from e

# Load data (will raise clear errors if missing or malformed)
agents = load_and_validate('agents.csv', ['agent_id', 'agent_name', 'region', 'onboarding_date', 'license_type', 'performance_score', 'target_growth'])
policies = load_and_validate('policies.csv', ['policy_id', 'agent_id', 'region', 'policy_value', 'premium', 'coverage_type', 'claim_frequency', 'avg_claim_severity', 'reinsurance_coverage', 'inception_date', 'expiry_date'])
events = load_and_validate('disaster_events.csv', ['event_id', 'event_type', 'region', 'date', 'severity', 'notes'])
print("All data loaded and validated.")

# Quick preview
display(
    agents.head(2),
    policies.head(2),
    events.head(2)
)

# Optional: Check for missing values
print("Missing values check:")
display(
    agents.isna().sum(),
    policies.isna().sum(),
    events.isna().sum()
)

## 3. Catastrophe Impact Simulation

In [ ]:
# --- User Input: Select Event to Simulate ---
simulate_event_id = 'E001'  # Default: simulate the first event; override as needed
event = events[events['event_id'] == simulate_event_id].iloc[0]
print(f"Simulating Event: {event['event_id']} | {event['event_type']} in {event['region']} (Severity: {event['severity']})\n")

In [ ]:
# --- Filter Affected Policies ---
region = event['region']
affected = policies[policies['region'] == region].copy()
if len(affected) == 0:
    print("WARNING: No policies found in the affected region.")
    print(f"Region: {region}")
    affected = pd.DataFrame(columns=policies.columns)  # Empty, but keep structure

# --- Estimate Payouts ---
# Gross payout: policy_value * avg_claim_severity * severity_factor
#             (where severity_factor depends on the event severity)
severity_factor = 0.25 * event['severity']  # Adjust as needed per business logic
affected['gross_payout'] = affected['policy_value'] * affected['avg_claim_severity'] * severity_factor

# Net payout after reinsurance: gross_payout * (1 - reinsurance_coverage)
affected['net_payout'] = affected['gross_payout'] * (1 - affected['reinsurance_coverage'])

# --- Compute Aggregate Metrics ---
total_gross = affected['gross_payout'].sum()
total_net = affected['net_payout'].sum()
reserves = 300_000_000  # Mock total reserves
solvency_ratio = reserves / total_net if total_net > 0 else np.inf

# --- Print Summary ---
print("Catastrophe Impact Summary:")
print(f"Event:           {event['event_type']} in {event['region']}")
print(f"Date:            {event['date']}")
print(f"Severity:        {event['severity']}")
print(f"Policies Impacted:   {len(affected)}")
print(f"Total Gross Payout:  ${total_gross:,.0f}")
print(f"Total Net Payout:    ${total_net:,.0f}")
print(f"Reserve Adequacy (Solvency Ratio): {solvency_ratio:.2f}\n")

## 4. Visualization & Business Insights

In [ ]:
if not affected.empty:
    # --- Policy Payout Distribution ---
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(affected['gross_payout'], bins=20, kde=True)
    plt.title('Gross Payout per Policy ($)')
    plt.xlabel('Gross Payout')
    plt.subplot(1, 2, 2)
    sns.histplot(affected['net_payout'], bins=20, kde=True)
    plt.title('Net Payout per Policy (after reinsurance) ($)')
    plt.xlabel('Net Payout')
    plt.tight_layout()
    plt.show()

    # --- Coverage Type Impact ---
    plt.figure(figsize=(8, 4))
    sns.barplot(
        data=affected.groupby('coverage_type')['gross_payout'].sum().reset_index(),
        x='coverage_type', y='gross_payout'
    )
    plt.title('Gross Payout by Coverage Type')
    plt.xticks(rotation=45)
    plt.show()

    # --- Top Agents by Impact ---
    affected_w_agents = affected.merge(agents, on='agent_id', how='left')
    top_agents = (
        affected_w_agents.groupby('agent_name')['gross_payout'].sum()
        .sort_values(ascending=False)
        .head(5)
        .reset_index()
    )
    plt.figure(figsize=(8, 4))
    sns.barplot(data=top_agents, x='agent_name', y='gross_payout')
    plt.title('Top 5 Agents by Gross Payout Impact')
    plt.xticks(rotation=45)
    plt.show()

## 5. Export for Power BI and Dashboards

In [ ]:
# --- Save Detailed Results for Power BI/Dashboard ---
affected.to_csv(
    os.path.join(OUTPUT_DIR, 'output_catastrophe.csv'),
    index=False
)
print(f"\nDetailed results saved to {OUTPUT_DIR}output_catastrophe.csv for Power BI integration.")

## 6. Documentation & Reproducibility

**Data Sources:**  
- `agents.csv`: Agent metadata, 50+ agents.  
- `policies.csv`: Policy portfolio, 50+ policies.  
- `disaster_events.csv`: Catastrophe log, 50+ events.  

**Output:**  
- `output_catastrophe.csv`: Per-policy simulation results (gross/net payout, coverage type, agent, region).  

**Business Logic:**  
- **Gross payout** = policy_value × avg_claim_severity × (0.25 × event_severity)  
- **Net payout** = gross_payout × (1.0 - reinsurance_coverage)  
- **Solvency ratio** = total_reserves / total_net_payout  

**Configurable:**  
- Set `simulate_event_id` to any `event_id` from `disaster_events.csv` to simulate alternate scenarios.  
- Adjust `severity_factor` formula as needed for your actuarial model.  
- `reserves` is arbitrarily set; replace with actual insurer reserve data.  

**Reproducible:**  
Run for any new event, data refresh, or region. Results are transparent and auditable.  

**Ready for Production:**  
- Data validation.  
- Clear error messages.  
- Visuals for executives.  
- Export for dashboards (Power BI/Tableau/Looker).  

**Next Steps:**  
- Batch or parallelize for multiple events.  
- Add Monte Carlo simulations for uncertainty.  
- Integrate with live catastrophe feeds.

## 7. Footnotes

- **Adjust** `simulate_event_id` to rerun for different catastrophes.  
- **Replace** `reserves` with your actual insurer reserve data when available.  
- **Extend** to batch mode per your needs.  
- **Power BI:** Connect to `output_catastrophe.csv` for live dashboards.  
- **For production:** Schedule with Airflow, GitHub Actions, etc.